In [ ]:
# 🧠 Fine-tuning LoRA - David WhatsApp Clone

# ✅ Étape 1 : Installer les dépendances
!pip install -q peft bitsandbytes accelerate transformers datasets trl

# ✅ Étape 2 : Importer les bibliothèques
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training, TaskType
from datasets import load_dataset, Dataset
from trl import SFTTrainer
import torch
import json

# ✅ Étape 3 : Uploader ton dataset (format HuggingFace style)
from google.colab import files
uploaded = files.upload()

# ✅ Étape 4 : Charger le dataset local
with open("david_conversations.json", "r") as f:
    data = json.load(f)

all_rows = []
for convo in data:
    for i in range(1, len(convo['conversations']), 2):
        user_msg = convo['conversations'][i-1]['content']
        david_msg = convo['conversations'][i]['content']
        prompt = f"<|user|>\n{user_msg}\n<|assistant|>\n{david_msg}"
        all_rows.append({"text": prompt})

dataset = Dataset.from_list(all_rows)

# ✅ Étape 5 : Charger le modèle (Zephyr ou Mistral)
model_id = "mistralai/Mistral-7B-Instruct-v0.1"
tokenizer = AutoTokenizer.from_pretrained(model_id, use_fast=True)
model = AutoModelForCausalLM.from_pretrained(model_id, load_in_4bit=True, torch_dtype=torch.float16, device_map="auto")

# ✅ Étape 6 : Appliquer LoRA
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.CAUSAL_LM,
)
model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, lora_config)

# ✅ Étape 7 : Configurer l'entraînement
training_args = TrainingArguments(
    output_dir="david-lora-output",
    num_train_epochs=3,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    logging_steps=10,
    learning_rate=2e-4,
    fp16=True,
    save_strategy="epoch",
    report_to="none",
)

# ✅ Étape 8 : Lancer le fine-tuning
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    tokenizer=tokenizer,
    args=training_args,
    packing=True,
    dataset_text_field="text",
)

trainer.train()

# ✅ Étape 9 : Sauvegarder dans Google Drive (si monté)
# from google.colab import drive
# drive.mount('/content/drive')
# model.save_pretrained("/content/drive/MyDrive/david-lora-model")
